Hi David,

The recollection of this morning's discussion:

Right now, you compute, for each crosscorrelation, an initial estimate of the combined timing error by crosscorrelating the first crosscorrelation with the last crosscorrelation. That is, even if there would not be a single interferometric surface wave response (not a high peak) in that crosscorrelation. 

In order to prevent this, we should first estimate the SNR of the causal and acausal peaks. You can call my routine for this, with the initial timing error estimate set to zero. For each station couple, you subsequently determine which crosscorrelations (i.e., associated with different crosscorrelation periods) have at least one of these peaks exceeding the SNR threshold. For each station couple, you then compute (from the crosscorrelations that remain) an initial drift rate based on the crosscorrelation of the first and last crosscorrelation. If there is only crosscorrelation for a station couple (i.e., only for a single time), you can of course not do this. All in all, this should, to some extend, prevent "bad" initial timing errors.

After you have the initial drift rates, you can use those to compute the data vector (i.e., the T_{app}) for solving the inverse problem. The conditions (SNR threshold, station separation, minimum number of crosscorrelations, etc.) of course need to be fulfilled. This data vector (and associated matrix a) can be used to invert for a first set of a's and b's. Note that you do not include the crosscorrelations in this first inversion for which you did not determine an initial drift rate.


 
In summary, the workflow/approach:
 

 
1. Set the following parameters:
1. a. SNR threshold
1. b. Minimum station separation
1. c. Minimum number of crosscorrelations
1. d. Minimum number of connections
1. e. Minimum number of crosscorrelation periods
1. f. Minimum separation in days
2. Compute causal and acausal SNR's, and station-station distances for all station couples.
3. Compute drift rates using the first and last crosscorrelation of a station couple (both these crosscorrelations should have one peak that exceeds the SNR threshold; see above)
4. Including only those station couples that fulfill the set conditions, and for which drift rates have been estimated, solve the inverse problem for the first time. You use the drift rates to estimate the expected timing errors for each of the crosscorrelations.
5. Use the obtained a's and b's to compute estimates of each crosscorrelation's time shift and again solve the invers problem (2nd iteration). In this case, you use all crosscorrelations that fulfil the conditions. 
6. Iterate a few times, until a's and b's are stable. 
7. Apply bootstrapping (use 1000 realizations) using an ordinary least squares approach, and weighted least squares approach. For all realizations, make sure the conditions are fulfilled. This will mean that number of rows and number of estimated a's and b's (i.e., number of stations) may vary between realizations
8. For each individual station, compute a minus the average a. That is, the average over 1000 (or a few less) realizations.
9. Make a histogram using all realizations and all stations (maximum 1000 x # of stations; in practice a bit less).
10. Do the same for the b values.
11. Fit a gaussian to the histograms, and determine the standard deviation (or variance). Note that you now again have to compute the average a and b, as the number of stations per realizations may vary.

Investigating dependency on different sets of parameters:

The workflow above, you should execute for different combinations of parameters. I propose the following two variations for the crosscorrelation conditions.

1. Minimum number of crosscorrelations for a station = 3, Minimum number of connections for a station = 2, Minimum number of crosscorrelation periods for a station = 2, minimum number of separation in days for a station's first and last crosscorrelation = 30 (this does not need to be with the same other station).
2. Minimum number of crosscorrelations for a station = 5, Minimum number of connections for a station = 3, Minimum number of crosscorrelation periods for a station = 3, minimum number of separation in days for a station's first and last crosscorrelation = 30 (this does not need to be with the same other station)

I propose you vary SNR between:
10
20
30
40
50
60

I propose you vary station-station separation threshold between:
1.5
2.0
2.5
3.0
3.5
4.0

The above means that you will execute the bootstrapping 2 x 6 x 6 = 72 times. Simply run it, and evaluate the results. Without bothering too much about the outcome. We will discuss the results coming Tuesday. We can then also decide for which specific combination we will do the bootstrapping again, but this time using SNR as weighting factor. After that, we are done in my opinion.

If things are unclear, please let me know!


In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 10 14:14:04 2022

@author: davidnaranjo
"""
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
# Importing the m importlibain code.
module_path = os.path.abspath('/Users/localadmin/Dropbox/GitHub/ocloc/src/ocloc')

if module_path not in sys.path:
    sys.path.append(module_path)
import ocloc
from ocloc import ProcessingParameters, ClockDrift, trim_correlation_trace
import ocloc
import pickle

# Parameters for locating the files where the correlation files and station 
# information is contained.
path2data_dir = "/Users/localadmin/Dropbox/GitHub/data"
# path2data_dir = "/Users/localadmin/Dropbox/GitHub/ocloc/tutorials/correlations_O20"
# station_file = "/Users/localadmin/Dropbox/GitHub/ocloc/tutorials/station_info"
station_file = "/Users/localadmin/Dropbox/GitHub/ocloc/tutorials/metadata/station_file.txt"

reference_time = '2014-08-21T00:00:00.000000Z'

# Investigating dependency on different sets of parameters:

1. min_number_of_total_correlations=3
2. min_number_correlation_periods=2
3. min_number_of_stationconnections=2
4. days_apart=30

In [3]:
min_number_of_total_correlations=3
min_number_correlation_periods=2
min_number_of_stationconnections=2
days_apart=30

### Varying SNR and dist_trh

In [6]:
for snr_trh in [10, 20, 30, 40, 50, 60]:
    for dist_trh in [1.5, 2.0, 2.5, 3.0, 3.5, 4.0]:
        params = ProcessingParameters(
            freqmin=0.2,   # Low freq. for the bandpass filter
            freqmax=0.4,   # High freq. for the bandpass filter 
            ref_vel=2500,  # m/s
            dist_trh=dist_trh,  # Minimum station separation in terms of wavelength
            snr_trh=snr_trh    # Signal-to-noise ratio threshold
            )

        cd = ClockDrift(station_file, path2data_dir, 
                        reference_time=reference_time,
                        list_of_processing_parameters=[params])

        cd.calculate_aprioridt_4_allcorrelations()
        cd.calculate_tapp_4_allcorrelations()

        for i in range(5):
            cd.filter_stations()
            cd.build_matrices()
            cd.solve_eq(method="weighted_lstsq")
            cd.calculate_dt_ins()
            cd.calculate_tapp_4_allcorrelations()

        from ocloc import suppress_stdout
        from statistics import mean
        import numpy as np
        import ocloc
        average_dates = [c.average_date for c in cd.correlations]
        mean_a_weighted = []
        mean_b_weighted = []
        mean_a_lstsq    = []
        mean_b_lstsq    = []
        #with suppress_stdout():
        for i in range(1000):
            bootstrapped_cd = cd.copy()
            correlations_with_tapp = []
            for c in bootstrapped_cd.correlations:
                if not np.isnan(c.t_app[-1]):
                    correlations_with_tapp.append(c)
            index_list = np.random.choice(range(len(correlations_with_tapp)),
                                          replace=True,
                                          size=len(correlations_with_tapp))

            res_list = [correlations_with_tapp[i] for i in index_list]
            bootstrapped_cd.correlations = res_list
            a = bootstrapped_cd.filter_stations(min_number_of_total_correlations,
                                                min_number_correlation_periods, 
                                                min_number_of_stationconnections,
                                                days_apart)

            with suppress_stdout():
                weighted_cd = bootstrapped_cd.copy()
                weighted_cd.calculate_dt_ins()
                weighted_cd.build_matrices()
                weighted_cd.solve_eq(method='weighted_lstsq')
                lstsq_cd = bootstrapped_cd.copy()
                lstsq_cd.calculate_dt_ins()
                lstsq_cd.build_matrices()
                lstsq_cd.solve_eq(method='lstsq')

            a_vals_weighted = []
            b_vals_weighted = []
            a_vals_lstsq    = []
            b_vals_lstsq    = []
            for station_w, station_l in zip(weighted_cd.stations,
                                            lstsq_cd.stations):

                if station_w.needs_correction:
                    a_vals_weighted.append(station_w.a[-1])
                    b_vals_weighted.append(station_w.b[-1])
                if station_l.needs_correction:
                    a_vals_lstsq.append(station_l.a[-1])
                    b_vals_lstsq.append(station_l.b[-1])
            if abs(mean(a_vals_weighted)) > 0.008:
                raise
            if abs(mean(a_vals_lstsq)) > 0.008:
                raise
            mean_a_weighted.append(mean(a_vals_weighted))
            mean_b_weighted.append(mean(b_vals_weighted))
            mean_a_lstsq.append(mean(a_vals_lstsq))
            mean_b_lstsq.append(mean(b_vals_lstsq))

        import matplotlib.pyplot as plt
        plt.hist(mean_a_weighted, bins = 50,
                alpha=0.5,
                facecolor="C1",
                edgecolor="black",
                linewidth=1
                )
        plt.title("Bootstrapped test of the weighted lstsq inversion")
        plt.xlabel("Mean a values using weighted_lstsq")
        plt.ylabel("Counts")
        # plt.xlim(-0.005, 0)
        # plt.ylim(0, 80)
        plt.show()

        plt.hist(mean_a_lstsq, bins = 50,
                alpha=0.5,
                facecolor="C1",
                edgecolor="black",
                linewidth=1
                )
        plt.title("Bootstrapped test of the lstsq inversion")
        plt.xlabel("Mean a values lstsq")
        plt.ylabel("Counts")
        # plt.xlim(-0.005, 0)
        # plt.ylim(0, 10)
        plt.show()

No correlation file found for station:#STK
Calculating the apriori estimates for each stationpair
Calculating the t_app for each stationpair.
Calculating a and b for each station.
The weighting is done based on the station separation.
Calculating the t_app for each stationpair.
Calculating a and b for each station.
The weighting is done based on the station separation.
Calculating the t_app for each stationpair.
Calculating a and b for each station.
The weighting is done based on the station separation.
Calculating the t_app for each stationpair.


KeyboardInterrupt: 

1. Minimum number of crosscorrelations for a station = 5
2. Minimum number of connections for a station = 3
3. Minimum number of crosscorrelation periods for a station = 3
4. minimum number of separation in days for a station's first and last crosscorrelation = 30

In [5]:
min_number_of_total_correlations=5
min_number_correlation_periods=3
min_number_of_stationconnections=3
days_apart=30